# Monte Carlo 2D Ising Model

Authors: Chris King, James Grant

This tutorial aims to help solidify your understanding of the theory underlying the Monte Carlo simulation technique by applying it to model the magnetic properties of a 2D material.  

In [5]:
# import everything that we will need in this tutorial now
import numpy
import matplotlib.pyplot as plt
from inputs.Tut_2.sources.ising import IsingModel
from inputs.Tut_2.sources.isingdata import IsingModelData

## Introduction to Monte Carlo Methods:

Monte Carlo is the name given to the simulation technique that attempts to solve a problem by randomly sampling out of all of its possible outcomes ('configurational space')and obtaining a result based on numerical analysis of the sampling.  Monte Carlo is a stochastic method, which means that the final state of the system cannot be predicted precisely based on the initial state and parameters, but through numerical analysis, reproducible results can be obtained.  

This contrasts with other techniques like molecular dynamics, which are deterministic, in that if you know the initial state and the inputs for the equations, you can predict what the configuration of the system will be at any and all times thereafter.  Based on this difference, Monte Carlo is used in a variety of applications across science where deterministic techniques are ineffective or impossible to use, such as phase co-existence and criticality, adsorption, and development of solid-state defects [1].

Results from Monte Carlo simulations are generally accurate and reliable, assuming that the technique has representatively sampled the distribution of possible configurations in the system ('configurational space').  As such, the choice of sampling method is crucial when formulating the parameters and criteria of your simulation.

There are many possible ways one can sample the configurational space of a simulated system, the intuitive case is simple random sampling in that we move randomly from one configuration to another and collect a representative sample of configurations that way.  However, the reliabilty of this process is heavily dependent on the probability distribution of possible states and does not take into account the respective weighting of a given configuration.  For example, it can under-represent a small number of configurations who contribute significantly to the overall state of the system and sample mostly insignificant states instead.

The concept of statistical weight of a configuration in a system is crucial in thermodynamics and describes how likely the particular configuration is of being observed out of a hypothetically *large* number of replicas of that system (an ensemble).  

For instance, consider the possible configurations of the gas molecules in this room, clearly, this system would have a high probability of being in a configuration where the gas molecules are evenly (on average) distributed throughout the volume of the room and so this configurational has a high weighting.  However, there is a configuration where every gas molecule sits in one corner of the room, this configuration is highly unlikely to be seen and so its weighting would be lower. The basic random sampling method would not take these weightings into account, so is often not used to sample configurational space unless all configurations are equally weighted.

There are more sophisticated ways of sampling configurational space, such as the Metropolis Algorithm, which is one of the most widely used sampling schemes in Monte Carlo simulations (including this one).  This algorithm proposes a move through configurational space (in this case, a spin flip) and calculates the energy of the two configurations (initial and final).  It then applies the following condition to accept/reject the move:

$$P_{\mathrm{acc}}(\mathbf{r}_1 \rightarrow \mathbf{r}_2) = \min(1, \exp \{- \beta [U(\mathbf{r}_2) - U(\mathbf{r}_1)] \} )$$

where $P_{\mathrm{acc}}(\mathbf{r}_1 \rightarrow \mathbf{r}_2)$ is the probability of accepting the move from the initial configuration, $\mathbf{r}_1$, with an energy, $U(\mathbf{r}_1)$, to the new configuration, $\mathbf{r}_2$, with an energy, $U(\mathbf{r}_2)$.  The function min() means that the smallest value in the brackets is chosen.  If the energy of the new configuration is less than that of the original, *i.e.* $U(\mathbf{r}_2) < U(\mathbf{r}_1)$, then $U(\mathbf{r}_2)-U(\mathbf{r}_1) < 0$ and so $\exp \{- \beta [U(\mathbf{r}_2)-U(\mathbf{r}_1)] \}  > 1$ and so the move is accepted with $P_{\mathrm{acc}}(\mathbf{r}_1 \rightarrow \mathbf{r}_2) = 1$.  If the new energy is greater than the energy of the original configuration, *i.e.* $U(\mathbf{r}_2) > U(\mathbf{r}_1)$, then $U(\mathbf{r}_2)-U(\mathbf{r}_1) > 0$ and so $\exp \{- \beta [U(\mathbf{r}_2) - U(\mathbf{r}_1)] \}  > 1$ and the move is accepted with probability $P_{\mathrm{acc}}(\mathbf{r}_1 \rightarrow \mathbf{r}_2) = \exp \{- \beta [U(\mathbf{r}_2) - U(\mathbf{r}_1)] \} < 1$.  

*N.B.* even if the proposed move leads to a higher-energy configuration, there is still a non-zero probability of it being accepted! Why should this be the case?

In [10]:
a = input()

What happens to the total number of accepted moves in a given simulation as we change the temperature? How might this affect the final outcome of your simulation?

In [11]:
b = input()

## Part 1: Ising Model of Magnetism

An application where Monte Carlo is more effective than deterministic methods is magnetic behaviour of solid state materials.  

Our simulation will be based on a 2D Ising model, which describes the macroscopic magnetic behaviour of a solid material as a result of the relative orientation of electron spins within the crystal lattice of a material.  As you may recall, each electron has an intrinsic 'spin'.  In simple terms, the spin of an electron can be thought of as a magnetic moment, with two possible orientations: 'up' and 'down'.  This idea helps define two classes of magnetic materials: diamagnetic and paramagnetic.  

Diamagnetic materials are made up of atoms/molecules without unpaired electrons, do not interact with external magnetic fields, making them non-magnetic.  Paramagnetic materials contain unpaired electrons, exhibiting a net magnetic moment that can interact with external magnetic fields and give the material its magnetic properties.  Figure 1 shows an example of a paramagnetic material as a 2D lattice of colour-coded spins.

<img src="images/Tut_2_images/paramagnet_config.png" />

**Figure 1:** A 2D schematic of a paramagnetic material under an external magnetic field.  Yellow indicates the spins that are aligned with the field and purple are spins that are anti-aligned.

There is another type of magnetism observed known as ferromagnetism, where instead of a uniform alignment of spins as in paramagnetic materials, 'domains' of aligned spins form, bound by domains of oppositely aligned spins (see Figure 2).  Ferromagnetic materials can show unique properties, such as being able to generate their own magnetic field (magnetisation) in the absence of an external magnetic field.  These materials are the common magnets seen in real-world applications.

<img src="images/Tut_2_images/ferromagnet_cand2.png" />

**Figure 2:** A 2D schematic of a ferromagnetic material at $T < T_{c}$.  Yellow and purple represent the two different spin orientations, 'up' and 'down', respectively.

The main factor influencing whether a given atom's spin is aligned with its neighbours in a crystal, and hence what type of magnetism the material displays, is its exchange energy, *E*, which in the Ising model is given by:

$$E = -J \sum_{<i,j>} s_{i}s_{j}$$

where *J* is the coupling constant between adjacent atoms in a given material and $s_{i/j}$ is the spin of the particle in position i/j in the lattice, respectively.  The <...> here mean the sum goes over the nearest neighbours of the atom in position (i,j), *i.e.* over the atoms at positions  (i-1, j), (i+1, j), (i, j-1) and (i, j+1) only.  The sign of *J* determines whether spin alignment (ferromagnetism) or anti-alignment (antiferromagnetism-see extension) is favourable.

The exchange energy can be thought of as an activation barrier for an atom to change its spin depending on the spins of its neighbours.  This means that, like with any physical system with an energy barrier, spontaneous thermal fluctuations can overcome the barrier and cause some atoms/domains to flip their spin, with the likelihood of flipping a spin increasing as temperature increases.  Therefore, ferromagnetic materials only show domains at temperatures under a specific critical, or Curie, temperature, $T_{c}$.  

Above this point, ferromagnetic materials lose their ability to retain magnetisation because the thermal fluctuations are much larger than the energy required to switch a domain's alignment with respect to other domains.  This results in a loss of the domain structure, and hence loss of magnetisation without an external field.  It is for this reason that paramagnetism can be thought of as high-temperature ferromagnetism.

For more information on the Ising model, consult either [2] or [3].

### Exercise 1)

The aim of this exercise is to familiarise yourself with running calculations on a simple 2D Ising model of a ferromagnetic material. The material is represented by a 64x64 2D lattice of points, each representing an atom with its own net spin.  In this exercise, all atoms are spin-aligned.  We will be running a Monte Carlo simulation to look at how the overall spin alignment (magnetisation) and energy of the system evolves with both time and temperature.

First, we shall setup our intial simulation at a given temperature:

In [2]:
data = dlmonte.DLMonteData("")

Now let's run our first Monte Carlo simulation of the day!

In [3]:
# Run the initial simulation.  Takes about a minute to complete

If you wish, you can look in your directory and see several new files have appeared.  The nature of these files will be explained in detail next session.  

Now that you have all the output data you could possibly need from this calculation, we shall proceed with extracting the time evolution of magnetisation and the distribution of the magnetisations over the course of the simulation. 

In [9]:
# output data extraction and analysis into plots of magnetisation vs time and histogram of magnetisation distributions

T = 2.36
plt.figure()
plt.subplot(1,2,1)
plt.xlabel("Number of steps")
plt.ylabel("Magnetisation")
plt.title("Time evolution of magnetisation at T = {}".format(T))
plt.axis()
plt.plot(M_seq.dat, 'b-')

plt.subplot(1,2,2)
plt.xlabel("M")
plt.ylabel("P(M)")
plt.title("Distribution of magnetisations at T = {}".format(T))
plt.hist(M_seq.dat, bins=auto, normed=True, weights=None)

NameError: name 'M_seq' is not defined

You will find several new files in your directory, but we will have used only on M_seq.dat and M_hist.dat in this exercise (we will get to the others later).

We shall now proceed to run the calculation at higher temperatures to obtain the temperature-dependence of the magnetisation.  Repeat the simulation and analysis sections that you have done for this initial temperature with the other temperatures in the main directory.

Compare the evolution of magnetisation as the temperature changes and rationalise any observed trends using your knowledge of ferromagnetism.  Do the results correspond to the Ising model?

In [1]:
c = input()

Compare the shapes of your magnetisation histograms as the temperature changes.  What does this indicate is happening to your system as temperature changes? Does this behaviour support the Ising model and your magnetisation evolution data?

In [2]:
d = input()

Once you have done that, plot magnetisation vs temperature for the system.  Comment on the shape of your graph and estimate the critical temperature, $T_{c}$, from it. 

In [3]:
e = input()

In [5]:
# collate all magnetisation-temperature data and plot it

For any square 2D Ising model where coupling along rows and along columns are equal, $T_{c}$ is given by:

$$T_{c} = \frac{2}{\ln(1+\sqrt{2})} \approx 2.269$$

Does your estimation of $T_{c}$ agree with that predicted by the above equation? Account for any observed discrepancies.  How could you improve the accuracy of your estimated value for $T_{c}$?

In [4]:
f = input()

### Extension (optional):

You have seen what happens as the system is heated, but you can also look at the magnetisation upon cooling the system from a state above the critical temperature to a state below the critical temperature. 

Go back to the beginning of Exercise 1 and now choose the inputs in ------- and plot the time-evolution of magnetisation.

How does this compare with the time evolution at $T>T_{c}$? Does this agree with the Ising model? If not, what do you think might be the problem with our simulation?

In [5]:
g = input()

### Exercise 2)

This exercise will demonstrate the stochastic nature of Monte Carlo simulation as well as how the Metropolis algorithm produces reliable and accurate results for this simple 2D Ising model.

We have seen what happens when we start the simulations from a fixed starting configuration (all spins aligned), but what will happen when we set the initial configuration to random? 

Go back the the beginning of Exercise 1 and repeat it for each temperature in the 'ranseed' folder, plotting the magnetisation vs. temperature once you have run all the simulations.  

How do the results from this exercise compare with those of Exercise 1? What effect does the initial configuration have on the outcome of the simulation?

In [6]:
h = input()

### Extension (optional):

For one of the ranseed calculations, let us find out what the initial configuration was and use that as our fixed starting configuration and see how the results of both calculations compare:

Running this command should return a line containing four integer numbers.  Create a new directory and copy the CONFIG, CONTROL and FIELD files into it.  Then, go to your CONTROL file and replace 'ranseed' with 'seeds int1 int2 int3 int4' where 'int' are the numbers from the command line in that order.

Re-run the calculation with this CONTROL file and plot the magnetisation vs time.  Compare this with the equivalent 'ranseed' calculation data.  

What do you notice about the magnetisation evolution in the two calculations? Does this confirm that the stochastic nature of Monte Carlo methods can produce reliable results?

In [7]:
i = input()

## Conclusions:

Now that you have reached the end of this tutorial, you will hopefully have a better understanding of the Monte Carlo method  and the motivation for its use.  You have simulated the magnetic properties of a 2D material based on the Ising model and obtained:

- the temperature-dependence of magnetisation
- the evolution of magnetisation with time
- validation of the stochastic nature of Monte Carlo

In the next tutorial, you will be introduced to a general Monte Carlo program called DLMONTE and use it to model the thermal properties of a Lennard-Jones material.

## Extensions (optional):

**1.** 
You have already looked at how the magnetic behaviour of a ferromagnetic system changes over time and temperature, but there is another possible type of magnetism called antiferromagnetism, where the sign of the coupling constant, *J*, from equation 1 changes sign.  This means that it is now favourable for the spin of one atom to be opposed to the spin of its neighbours, resulting in a preferable 'checkerboard' pattern of magnetisation on the 2D lattice (see Figure 3).  You can investigate the magnetic behaviour in this case using the 2D Ising model.

<img src="images/Tut_2_images/antiferromagnet.png" />

**Figure 3:** The most stable magnetic configuration of an antiferromagnetic material at $T < T_{c}$.

Repeat Exercise 1 but this time using the inputs in the 'antiferromagnet' folder, plotting the temperature dependence of the magnetisation once you have run the simulation at each temperature.

Compare your results of the antiferromagnet with the ferromagnet.  Rationalise any observed differences in terms of exchange energy and alignment of spins.

In [8]:
j = input()

## References:

[1] S. Mordechai (Editor), *Applications of Monte Carlo Method in Science and Engineering* [Online]. Available: https://www.intechopen.com/books/applications-of-monte-carlo-method-in-science-and-engineering 

[2] J. V. Selinger, "Ising Model for Ferromagnetism" in *Introduction to the Theory of Soft Matter: From Ideal Gases to Liquid Crystals*.  Cham: Springer International Publishing, 2016, pp. 7-24.

[3] N. J. Giordano, *Computational Physics*.  Upper Saddle River, N.J.: Prentice Hall, 1997. 